<a href="https://colab.research.google.com/github/dgeorge1000/COVID-19-CNN-Image-Detection-from-Lung-X-Rays/blob/main/COVID19_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset #1: http://cb.lk/covid_19
# Dataset #2: https://www.dropbox.com/sh/7hrnecm53hk0ih3/AABsfKTooQP0Tc3QddOQAfZJa?dl=0

# download the dataset from the Dropbox link
!wget http://cb.lk/covid_19

In [ ]:
# unzip the folder to get each image for local access
!unzip covid_19

In [3]:
# paths for both the training dataset and the validation set
# training will be used for the CNN to learn how to differentiate between COVID and non COVID lung images
# validation will be used to test the CNN on data not seen to see its accuracy
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Val"

In [4]:
# various libraries and imports needed
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

In [5]:
# CNN Based Model using Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [6]:
# get the general information about the CNN created
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [7]:
# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [8]:
# using generator saves memory
train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')


Found 224 images belonging to 2 classes.


In [9]:
# 2 classifications
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [10]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 60 images belonging to 2 classes.


In [13]:
# CNN will begin training and testing on the given data
# "accuracy" and "val_accuracy" will show how accurate the CNN is
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=5,
    epochs = 8,
    validation_data = validation_generator,
    validation_steps=2
)

Performance shown below.

Epoch 1/8
5/5 [==============================] - 8s 2s/step - loss: 0.6761 - accuracy: 0.6000 - val_loss: 0.6456 - val_accuracy: 0.6167
Epoch 2/8
5/5 [==============================] - 7s 1s/step - loss: 0.6452 - accuracy: 0.6500 - val_loss: 0.6264 - val_accuracy: 0.9167
Epoch 3/8
5/5 [==============================] - 7s 2s/step - loss: 0.5789 - accuracy: 0.6750 - val_loss: 0.5282 - val_accuracy: 0.9500
Epoch 4/8
5/5 [==============================] - 7s 2s/step - loss: 0.4540 - accuracy: 0.7688 - val_loss: 0.3136 - val_accuracy: 0.9500
Epoch 5/8
5/5 [==============================] - 7s 2s/step - loss: 0.3411 - accuracy: 0.8687 - val_loss: 0.3142 - val_accuracy: 0.9333
Epoch 6/8
5/5 [==============================] - 7s 2s/step - loss: 0.2728 - accuracy: 0.9187 - val_loss: 0.1579 - val_accuracy: 0.9500
Epoch 7/8
5/5 [==============================] - 7s 2s/step - loss: 0.2119 - accuracy: 0.9375 - val_loss: 0.2037 - val_accuracy: 0.9833
Epoch 8/8
5/5 [==============================] -